# This is a script for preprocessing

# ERA5 2-m Air Temperature

In [1]:
parpool(10)

Starting parallel pool (parpool) using the 'Processes' profile ...
Connected to parallel pool with 10 workers.

ans = 

 ProcessPool with properties: 

            Connected: true
           NumWorkers: 10
                 Busy: false
              Cluster: Processes (Local Cluster)
        AttachedFiles: {}
    AutoAddClientPath: true
            FileStore: [1x1 parallel.FileStore]
           ValueStore: [1x1 parallel.ValueStore]
          IdleTimeout: 30 minutes (30 minutes remaining)
          SpmdEnabled: true



In [3]:
path='/glade/campaign/univ/ufsu0029/ERA5/t2m';

t2m=nan*zeros(360,181,365,40);
parfor m=1:40
    
    aa=ncread(append(path,'/t2m_',num2str(m+1980),'.nc'),'t2m');
    aa=reshape(aa,size(aa,1),size(aa,2),4,[]);
    aa=squeeze(mean(aa,3));
    if(size(aa,3)>365.5);aa(:,:,60)=[];end
    t2m(:,:,:,m)=aa;
    
end
size(t2m)

ans = 1x4 double
   360   181   365    40

In [7]:
lon=[0:359]';
lat=[90:-1:-90]';
t2m=single(t2m);
save('/glade/campaign/univ/ufsu0029/work/Data/ERA5/t2m_8020.mat','t2m','lon','lat','-v7.3');

# NCEP2 2-m Air Temperature

In [8]:
path='/glade/campaign/univ/ufsu0029/NCEP_DOE/t2m/';

air2m=nan*zeros(192,94,365,40);
parfor m=1:40
    
    aa=ncread(append(path,'/air.2m.gauss.',num2str(m+1980),'.nc'),'air');
    aa=reshape(aa,size(aa,1),size(aa,2),4,[]);
    aa=squeeze(mean(aa,3));
    if(size(aa,3)>365.5);aa(:,:,60)=[];end
    air2m(:,:,:,m)=aa;
    
end
size(air2m)

ans = 1x4 double
   192    94   365    40

In [9]:
lon=ncread(append(path,'/air.2m.gauss.',num2str(1+1980),'.nc'),'lon');
lat=ncread(append(path,'/air.2m.gauss.',num2str(1+1980),'.nc'),'lat');
air2m=single(air2m);
save('/glade/campaign/univ/ufsu0029/work/Data/NCEP_DOE/air2m_8020.mat','air2m','lon','lat','-v7.3');

# ERA5 Sea Ice Cover

In [10]:
path='/glade/campaign/univ/ufsu0029/ERA5/sic/';

sic=nan*zeros(360,181,365,40);
parfor m=1:40
    
    aa=ncread(append(path,'/sic_',num2str(m+1980),'.nc'),'siconc');
    aa=reshape(aa,size(aa,1),size(aa,2),4,[]);
    aa=squeeze(mean(aa,3));
    if(size(aa,3)>365.5);aa(:,:,60)=[];end
    sic(:,:,:,m)=aa;
    
end
size(sic)

ans = 1x4 double
   360   181   365    40

In [11]:
lon=ncread(append(path,'/sic_',num2str(1+1980),'.nc'),'longitude');
lat=ncread(append(path,'/sic_',num2str(1+1980),'.nc'),'latitude');
sic=single(sic);
save('/glade/campaign/univ/ufsu0029/work/Data/ERA5/sic_8020.mat','sic','lon','lat','-v7.3');

# ERA5 V Wind

In [20]:
path='/glade/campaign/univ/ufsu0029/ERA5/';

R=6370000;dp=10000;g=10;lat=[90:-1:-90];
v_n=nan*zeros(360,181,40);
v_s=nan*zeros(360,181,40);
parfor m=1:40
    
    aa0=ncread(append(path,'/vwnd/v_plev_',num2str(m+1980),'.nc'),'v');
    aa2=ncread(append(path,'/sp/sp_',num2str(m+1980),'.nc'),'sp');
    mask=(1+zeros(size(aa0))).*reshape(100*[950:-100:50],1,1,[]);
    aa0(mask>reshape(aa2,size(aa2,1),size(aa2,2),1,size(aa2,3)))=nan;

    aa1=aa0*dp/g;
    aa1(aa1>0)=nan;
    aa1=squeeze(mean(sum(aa1,3,'omitnan'),4));
    aa2=aa0*dp/g;
    aa2(aa2<0)=nan;
    aa2=squeeze(mean(sum(aa2,3,'omitnan'),4));
    
    v_s(:,:,m)=aa1;
    v_n(:,:,m)=aa2;
    
end
size(v_n)

ans = 1x3 double
   360   181    40

In [17]:
save('/glade/campaign/univ/ufsu0029/work/Data/ERA5/meridional_velocity.mat','v_n','v_s','lat'); %kg/m/s

# CESM2 Solar Irradaince & Temperature

In [18]:
path='/glade/campaign/univ/ufsu0029/eccentricity/';
file1=dir(append(path,'/B1850/B1850*'));
file2=dir(append(path,'/e000_lop000/e000_lop000*'));
file3=dir(append(path,'/e167_lop280/e167_lop280*'));

t2m_m=nan*zeros(144,96,365,50,3);
solar_m=nan*zeros(144,96,365,50,3);
parfor m=1:50

    bb=nan*zeros(144,96,365,1,3);
    aa=ncread(append(path,'/B1850/',file1(m).name),'TS');
    bb(:,:,:,:,1)=aa;
    aa=ncread(append(path,'/e000_lop000/',file2(m).name),'TS');
    bb(:,:,:,:,2)=aa;
    aa=ncread(append(path,'/e167_lop280/',file3(m).name),'TS');
    bb(:,:,:,:,3)=aa;
    t2m_m(:,:,:,m,:)=bb;
    
    bb=nan*zeros(144,96,365,1,3);
    aa=ncread(append(path,'/B1850/',file1(m).name),'SOLIN');
    bb(:,:,:,:,1)=aa;
    aa=ncread(append(path,'/e000_lop000/',file2(m).name),'SOLIN');
    bb(:,:,:,:,2)=aa;
    aa=ncread(append(path,'/e167_lop280/',file3(m).name),'SOLIN');
    bb(:,:,:,:,3)=aa;
    solar_m(:,:,:,m,:)=bb;

end

solar_m=squeeze(mean(solar_m,4));
lon3=ncread(append(path,'/B1850/',file1(1).name),'lon');
lat3=ncread(append(path,'/B1850/',file1(1).name),'lat');

In [19]:
lon=ncread(append(path,'/B1850/',file1(1).name),'lon');
lat=ncread(append(path,'/B1850/',file1(1).name),'lat');
t2m_m=single(t2m_m);
save('/glade/campaign/univ/ufsu0029/work/Data/eccen/solar_m.mat','solar_m','lon','lat');
save('/glade/campaign/univ/ufsu0029/work/Data/eccen/t2m_m.mat','t2m_m','lon','lat','-v7.3');